In [7]:
from glob import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import os.path as path
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer as count_vectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB as multinomial_nb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import load_digits
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
import string
import time
import operator
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Activation, Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
from matplotlib import colors as mcolors

ModuleNotFoundError: No module named 'nltk'

In [6]:
#Testing